In [46]:
import pandas as pd
from pandas.io import sql
import sqlite3
import re
import HTMLParser

In [29]:
conn = sqlite3.connect('ArticlesScraping.db')

sql.read_sql(
"""
select CleanedBody from ArticleCorpus
""", con=conn).head(1)

,CleanedBody
0,[\nTop stories\n\nFinancial Times: Royal Bank ...


In [14]:
#Need to add a line of code here to remove duplicates in table (based on url? Header? time?)

In [15]:
#Need to add a line of code here to clean up the Author column

In [16]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
# Remove stop words

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

# Make it a raw string
def raw_string(s):
    if isinstance(s, str):
        s = s.encode('string-escape')
    elif isinstance(s, unicode):
        s = s.encode('unicode-escape')
    return s

# Remove stop words
def remove_stopwords(content):
    cleaned = filter(lambda x: x not in ENGLISH_STOP_WORDS,content.split())
    return ' '.join(cleaned)

#remove punctuation
import string
def remove_punctuation(content):
    return filter(lambda x: x in string.ascii_letters+" ",content)

#remove numbers
def remove_numbers(content):
    return re.sub(r'\d+', '', content)

#remove some specific news/finance words that are killing the model
stopwords2 = ['million', 'billion', 'millions', 'billions', 'said', 'xa', 'nthe']
def remove_stopwords2(content):
    cleaned = filter(lambda x: x not in stopwords2,content.split())
    return ' '.join(cleaned)

In [17]:
def preprocess(s):
    s=s.lower()
    s= raw_string(s)
    #stem/lemmatize
    
    #remove
    s= remove_punctuation(s)
    s= remove_stopwords(s)
    s= remove_stopwords2(s)
    s= remove_numbers(s)
    return s

from sklearn.feature_extraction.text import CountVectorizer
c_vectorizer = CountVectorizer(preprocessor=preprocess,
                               stop_words='english',
                               min_df=.05)

count_matrix = c_vectorizer.fit_transform(
    sql.read_sql("SELECT CleanedBody from ArticleCorpus",con=conn)['CleanedBody']
    )
n_topics = 5
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=10)
lda.fit(count_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [18]:
data = sql.read_sql("SELECT FullLink, CleanedBody from ArticleCorpus",con=conn)
results = lda.transform(c_vectorizer.fit_transform(data['CleanedBody']))
topic_df = pd.DataFrame(results,columns=["Topic%s" % i for i in range(5)])
topic_df['FullLink'] = data['FullLink']
topic_df.to_sql(name='Article_Topics',
            con=conn,
            if_exists='replace',
            index=False)
topic_df.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,FullLink
0,176.189792,0.202712,0.203553,0.201999,0.201944,http://citywire.co.uk/wealth-manager/news/satu...
1,0.202835,0.202499,0.201410,40.191482,0.201774,http://citywire.co.uk/wealth-manager/news/rath...
2,0.200000,0.200000,0.200000,0.200000,0.200000,http://citywire.co.uk/wealth-manager/news/cent...
3,0.201718,0.203888,22.188804,0.202666,0.202924,http://citywire.co.uk/wealth-manager/news/weal...
4,46.102190,38.775066,15.712272,0.203787,0.206684,http://citywire.co.uk/wealth-manager/news/jupi...
5,0.202663,94.190370,0.202294,0.202774,0.201899,http://citywire.co.uk/wealth-manager/news/towr...
6,113.193732,0.201085,0.202577,0.201884,0.200722,http://citywire.co.uk/wealth-manager/news/shel...
7,0.203149,0.201284,75.191678,0.202656,0.201232,http://citywire.co.uk/wealth-manager/news/hedg...
8,0.204633,0.201837,89.187600,0.203299,0.202630,http://citywire.co.uk/wealth-manager/news/spec...
9,37.188226,0.201365,0.202447,0.203384,0.204578,http://citywire.co.uk/wealth-manager/news/auro...


In [19]:
n_top_words = 15
import operator
vocabulary = c_vectorizer.get_feature_names()
for i in range(n_topics):
    best_words_indexes = lda.components_[i].argsort()[:-n_top_words - 1:-1]
    best_words = " ".join([vocabulary[i] for i in best_words_indexes])
    print best_words

market times oil uk management companies trade brexit investors price stocks investment following platform month
jersey eu office financial business uk director service people opportunities advice client team private retirement
fund government income bond investors market likely rules sector uk result banks vote wealth yields
investment director new management fund wealth market investors firm manager assets months need company current
china mutual year returned company managed overall slowing nicholls trust markets manager asset confident net



Possible topics could be:
1. Investment Strategies
2. Wealth / D2C focused market
3. EU-centric Brexit topics
4. Macroeconomic concerns
5. Value/Equity investing


In [26]:
UserRatingsdf = pd.read_csv("UserRatings.csv",usecols=[1,2,3])
UserRatingsdf.head(2)
UserRatingsdf.to_sql(name='userratings.db',
            con=conn,
            if_exists='replace',
            index=False)
UserRatingsdf.head(2)

,FullLink,Rating,User
0,http://citywire.co.uk/wealth-manager/news/satu...,0,SamB
1,http://citywire.co.uk/wealth-manager/news/rath...,0,SamB


In [47]:
#remove them backslashes
def remove_backslashes(content):
    return re.sub(r'\\.', ' ', content)
def remove_morebackslashes(content):
    return re.sub(r'\.', ' ', content)
def remove_evenmorebackslashes(content):
    return re.sub(r'\..', ' ', content)
def remove_doublespace(content):
    return re.sub(r'  ',' ', content)

def thebigcleanup(content):
    content = remove_numbers(content)
    content = HTMLParser.HTMLParser().unescape(content)
    content = remove_backslashes(content)
    content = remove_morebackslashes(content)
    content = remove_evenmorebackslashes(content)
    content = remove_doublespace(content)
    return content

In [52]:
data = sql.read_sql(
"""
select * from ArticleCorpus
""", con=conn)
#Create new column 'Extra cleaned body' 
data["ExtraCleanedBody"] = data["CleanedBody"].apply(thebigcleanup)
data.head()

,PageLinks,FullLink,Time,Author,Header,Body,CleanedBody,ExtraCleanedBody
0,/wealth-manager/news/saturday-papers-rbs-to-ch...,http://citywire.co.uk/wealth-manager/news/satu...,2016-08-20 05:17:37,>HimanshuSingh],Saturday Papers: RBS to charge big clients for...,"[<div class=""article-body"" itemprop=""articleBo...",[\nTop stories\n\nFinancial Times: Royal Bank ...,[ Top stories Financial Times: Royal Bank of S...
1,/wealth-manager/news/rathbones-adds-to-london-...,http://citywire.co.uk/wealth-manager/news/rath...,2016-08-19 13:42:56,>JoshuaThurston],Rathbones adds to London office with second HS...,"[<div class=""article-body"" itemprop=""articleBo...",[\nRathbone Brothers has hired its second HSBC...,[ Rathbone Brothers has hired its second HSBC ...
2,/wealth-manager/news/central-banks-pouring-fue...,http://citywire.co.uk/wealth-manager/news/cent...,2016-08-19 12:52:45,>AndrewMilligan],Central banks pouring fuel on dying embers,[],[],[]
3,/wealth-manager/news/wealth-manager-olympics-t...,http://citywire.co.uk/wealth-manager/news/weal...,2016-08-19 12:27:35,[],Wealth Manager Olympics: the Relay,"[<div class=""article-body jsArticle-body--star...","[\nIf like us, the Wealth Manager Olympics has...","[ If like us, the Wealth Manager Olympics has ..."
4,/wealth-manager/news/jupiters-davies-buys-bomb...,http://citywire.co.uk/wealth-manager/news/jupi...,2016-08-19 11:21:00,>JonathanYarker],Jupiter's Davies buys 'bombed out' domestics,"[<div class=""article-body"" itemprop=""articleBo...","[\n\xa0Steve Davies, manager of the\xa0Jupiter...","[ aSteve Davies, manager of the aJupiter UK Gr..."


In [9]:
conn.close()